# Calculate the final grid box

- According to grid search results (See the hierachical clustering gri search notebook) Use the median clustering method with following parameters:

     - Standard factor (for the median filter): 100

     - pind_weight: 2.5

     - Clustering with Kmean

In [1]:
# Define data path
data_path = "/Users/nicha/dev/Protein-preparation-pipeline/data/toy_examples_clustering"

# System and OS utilities
import os
import sys

# Numerical and Data Processing
import numpy as np
import pandas as pd

# Scientific and Bioinformatics Tools
from math import e
from pymol import cmd, stored
import numpy as np
from Bio import PDB
from scipy.spatial import distance_matrix, KDTree
from scipy.stats import shapiro
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import combinations, product


# Machine Learning & Clustering
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import euclidean_distances

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import py3Dmol

# Color Mapping for Visualization
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.colors import Normalize
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.linalg import eigh
from scipy.sparse.csgraph import laplacian
from sklearn.neighbors import kneighbors_graph


# Add project-specific source path
sys.path.append('/Users/nicha/dev/Protein-preparation-pipeline/src/')

# Custom Modules from Your Project
from pdb_retrival.data_retriever import PDBDataRetriever

In [2]:
# Function for the data preparation

# 1️⃣ Extract residue coordinates from PDB
def extract_residue_coordinates(pdb_file, residue_number, chain_id="A"):
    """
    Extracts atomic coordinates for a given residue from a PDB file.

    Args:
        pdb_file (str): Path to the PDB file.
        residue_number (int): Residue number to extract.
        chain_id (str): Chain ID of the residue.

    Returns:
        list: List of tuples (atom_type, x, y, z).
    """
    coordinates = []
    try:
        with open(pdb_file, "r") as file:
            for line in file:
                if line.startswith(("ATOM", "HETATM")) and line[21] == chain_id:
                    resi = int(line[22:26].strip())
                    if resi == residue_number:
                        atom_type = line[76:78].strip()
                        x = float(line[30:38].strip())
                        y = float(line[38:46].strip())
                        z = float(line[46:54].strip())
                        coordinates.append((atom_type, x, y, z))
    except FileNotFoundError:
        print(f"Error: File {pdb_file} not found.")
    except Exception as e:
        print(f"Unexpected error: {e}")
    return coordinates


# 2️⃣ Compute Weighted Center of Mass
def calculate_weighted_center_of_mass(coordinates):
    """
    Calculates the weighted center of mass for a given set of atomic coordinates.

    Args:
        coordinates (list): List of tuples (atom_type, x, y, z).

    Returns:
        list: [x, y, z] coordinates of the weighted center of mass.
    """
    total_weight = 0
    weighted_coords = np.zeros(3)
    atomic_weights = {"H": 1.008, "C": 12.011, "N": 14.007, "O": 15.999, "S": 32.06}

    for atom_type, x, y, z in coordinates:
        weight = atomic_weights.get(atom_type.upper(), 1.0)
        weighted_coords += np.array([x, y, z]) * weight
        total_weight += weight

    return np.round(weighted_coords / total_weight, 3).tolist()


# 3️⃣ Process a Single PDB File and Update the DataFrame
def process_pdb_file(data_path, pdb_code, df):
    """
    Reads a PDB file and updates a DataFrame with residue center of mass.

    Args:
        data_path (str): Path to the directory containing the PDB and CSV files.
        pdb_code (str): PDB code of the protein.
        df (pd.DataFrame): DataFrame containing residue data.

    Returns:
        pd.DataFrame: Updated DataFrame with center of mass information.
    """
    pdb_file = os.path.join(data_path, f"{pdb_code}.pdb")

    try:
        for index, row in df.iterrows():
            residue_number = row["resi"]
            chain_id = row["chain"]
            coordinates = extract_residue_coordinates(pdb_file, residue_number, chain_id)
            if coordinates:
                center_of_mass = calculate_weighted_center_of_mass(coordinates)
                df.loc[index, "resn_coordinates"] = str(coordinates)
                df.loc[index, ["center_of_mass_x", "center_of_mass_y", "center_of_mass_z"]] = center_of_mass
            else:
                print(f"Residue {residue_number} in chain {chain_id} not found in {pdb_file}.")
    except FileNotFoundError:
        print(f"Error: {pdb_file} not found.")
    except Exception as e:
        print(f"Unexpected error: {e}")

    return df


# 4️⃣ Process and Update Data for a Single PDB Structure
def process_and_update_pdb_data(data_path, pdb_code):
    """
    Reads and processes a PDB file and corresponding CSV file to update residue information.

    Args:
        data_path (str): Path to the directory containing the PDB and CSV files.
        pdb_code (str): PDB code of the protein.

    Returns:
        pd.DataFrame: Updated DataFrame with residue information.
    """
    csv_file_path = f"{data_path}/results_{pdb_code}.csv"

    try:
        df = pd.read_csv(csv_file_path)
        df_updated = process_pdb_file(data_path, pdb_code, df)
        df_updated.to_csv(f"{data_path}/results_{pdb_code}_updated.csv", index=False)
        return df_updated
    except Exception as e:
        print(f"Error processing {pdb_code}: {e}")
        return None


# 5️⃣ Process All PDB Files in a Directory
def process_all_pdb_files(data_path):
    """
    Process all PDB-related files in the given directory and combine results.

    Args:
        data_path (str): Path to the directory containing the PDB and CSV files.

    Returns:
        pd.DataFrame: Combined DataFrame with all processed data.
    """
    combined_df = pd.DataFrame()

    for file in os.listdir(data_path):
        if file.startswith("results_") and file.endswith(".csv"):
            pdb_code = file.split("_")[1].split(".")[0]
            updated_df = process_and_update_pdb_data(data_path, pdb_code)
            if updated_df is not None:
                updated_df["PDBcode"] = pdb_code
                combined_df = pd.concat([combined_df, updated_df], ignore_index=True)

    combined_df.drop_duplicates(inplace=True)
    combined_df.to_csv(f"{data_path}/combined_results.csv", index=False)
    return combined_df


# 6️⃣ Compute Center of Protein Using PyMOL
def calculate_center_of_mass_pymol(pdb_file):
    """
    Calculates the center of mass of a protein using PyMOL.

    Args:
        pdb_file (str): Path to the PDB file.

    Returns:
        tuple: (x, y, z) coordinates of the protein's center of mass.
    """
    cmd.load(pdb_file, "protein")
    center_of_mass = cmd.centerofmass("protein")
    cmd.delete("all")
    return round(center_of_mass[0], 3), round(center_of_mass[1], 3), round(center_of_mass[2], 3)


# 7️⃣ Compute Protein Diameter
def calculate_protein_diameter(df):
    """
    Computes the diameter of a protein, defined as the maximum distance between residues.

    Args:
        df (pd.DataFrame): DataFrame containing residue data with center of mass coordinates.

    Returns:
        float: Maximum distance between residues.
    """
    coordinates = df[['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].values
    distances = np.linalg.norm(coordinates[:, np.newaxis] - coordinates, axis=2)
    return round(np.max(distances), 3)


# 8️⃣ Process Protein Data for a Single PDB
def process_protein_data(pdb_code, dfs, data_path):
    """
    Processes protein data by computing center of mass and diameter.

    Args:
        pdb_code (str): The PDB code of the protein.
        dfs (dict): Dictionary containing DataFrames indexed by PDB code.
        data_path (str): Path to the directory containing the PDB files.

    Returns:
        pd.DataFrame: Updated DataFrame with center of protein and diameter.
    """
    pdb_file_path = f"{data_path}/{pdb_code}.pdb"
    df = dfs[pdb_code]
    center_of_mass = calculate_center_of_mass_pymol(pdb_file_path)
    protein_diameter = calculate_protein_diameter(df)

    df["protein_x"], df["protein_y"], df["protein_z"] = center_of_mass
    df["protein_diameter"] = protein_diameter

    return df



def normalize_features(df, feature_columns):
    """
    Normalize or standardize features based on their distribution.

    - **Standardizes (Z-score) if normally distributed**.
    - **Normalizes (Min-Max) if not normally distributed**.
    - **Adds new columns instead of replacing existing ones**.
    - **Stores scalers for consistent transformation in clustering**.

    Args:
        df (pd.DataFrame): DataFrame containing features.
        feature_columns (list): List of feature column names (e.g., 3D coordinates).

    Returns:
        pd.DataFrame: DataFrame with added transformed feature columns.
        dict: Dictionary of fitted scalers (for later reuse).
    """
    processed_df = df.copy()
    scalers = {}

    for col in feature_columns:
        scaler = MinMaxScaler()  # Normalization
        new_col_name = f"{col}_normalized"

        processed_df[new_col_name] = scaler.fit_transform(df[[col]])  # Apply transformation
        scalers[col] = scaler  # Store scaler for reuse

    return processed_df, scalers


def weighted_pbind(df, pbind_column='p(bind)', weights=[1.0]):
    """
    Normalize and apply multiple weight factors to p(bind).

    - **Scales p(bind) between 0 and 1**.
    - **Applies multiple weight factors** to create additional columns.
    - **Adds new columns instead of replacing p(bind)**.
    - **Stores scaler** for reuse in clustering.

    Args:
        df (pd.DataFrame): DataFrame containing p(bind).
        pbind_column (str): Column name for p(bind).
        weights (list): List of weight multipliers (e.g., [1.0, 5.0, 10.0, 100.0]).

    Returns:
        pd.DataFrame: DataFrame with weighted p(bind) columns.
        dict: Dictionary of fitted scalers.
    """
    processed_df = df.copy()
    scalers = {}

    if pbind_column in df.columns:
        scaler = MinMaxScaler()
        normalized_pbind = scaler.fit_transform(df[[pbind_column]])  # Scale to 0-1
        scalers[pbind_column] = scaler  # Store scaler

        for weight in weights:
            weighted_col_name = f"{pbind_column}_weight_{weight}"
            processed_df[weighted_col_name] = normalized_pbind * weight  # Apply weight

    return processed_df, scalers
    """
    Normalize and apply multiple weight factors to p(bind).

    - **Scales p(bind) between 0 and 1**.
    - **Applies multiple weight factors** to create additional columns.
    - **Stores scaler** for reuse in clustering.

    Args:
        df (pd.DataFrame): DataFrame containing p(bind).
        pbind_column (str): Column name for p(bind).
        weights (list): List of weight multipliers (e.g., [1.0, 1.5, 2.0]).

    Returns:
        pd.DataFrame: DataFrame with weighted p(bind) columns.
        dict: Dictionary of fitted scalers.
    """
    processed_df = df.copy()
    scalers = {}

    if pbind_column in df.columns:
        scaler = MinMaxScaler()
        normalized_pbind = scaler.fit_transform(df[[pbind_column]])  # Scale to 0-1
        scalers[pbind_column] = scaler  # Store scaler

        for weight in weights:
            weighted_col_name = f"{pbind_column}_weight_{weight}"
            processed_df[weighted_col_name] = normalized_pbind * weight  # Apply weight

    return processed_df, scalers


In [3]:
# Function to evaluate the clustering performance
from numpy import size


def calculate_protein_center_of_mass(pdb_file):
    """
    Calculate the center of mass of the entire protein using PyMOL.

    Args:
        pdb_file (str): Path to the PDB file.

    Returns:
        tuple: Center of mass coordinates (x, y, z).
    """
    cmd.load(pdb_file, "protein")
    center_of_mass = cmd.centerofmass("protein")
    cmd.delete("all")  # Clear the loaded structure
    return tuple(round(coord, 3) for coord in center_of_mass)

def calculate_cluster_centers(df, cluster_column, pdb_file):
    """
    Calculate the center of mass for each cluster using PyMOL.

    Args:
        df (pd.DataFrame): DataFrame containing residue and cluster information.
        cluster_column (str): Column name containing cluster IDs.
        pdb_file (str): Path to the PDB file.

    Returns:
        dict: Cluster IDs as keys and their center of mass coordinates as values.
    """
    cmd.load(pdb_file, "protein")
    cluster_centers = {}
    clusters = df[cluster_column].unique()

    for cluster_id in clusters:
        cluster_data = df[df[cluster_column] == cluster_id]
        resi_selection = "+".join(map(str, cluster_data['resi']))
        cmd.select(f"cluster_{cluster_id}", f"resi {resi_selection}")
        center_of_mass = cmd.centerofmass(f"cluster_{cluster_id}")
        cluster_centers[cluster_id] = tuple(round(coord, 3) for coord in center_of_mass)
        cmd.delete(f"cluster_{cluster_id}")

    cmd.delete("all")  # Clear the loaded structure
    return cluster_centers

def calculate_ligand_centers_slow(pdb_file):
    """
    Calculate the center of mass for each ligand in the structure using PyMOL.

    Args:
        pdb_file (str): Path to the PDB file.

    Returns:
        dict: Ligand identifiers as keys and their center of mass coordinates as values.
    """
    cmd.load(pdb_file, "protein")
    cmd.select("ligands", "organic")
    ligand_centers = {}

    for atom in cmd.get_model("ligands").atom:
        ligand_id = f"{atom.chain}_{atom.resn}_{atom.resi}"
        cmd.select("ligand", f"chain {atom.chain} and resn {atom.resn} and resi {atom.resi}")
        center_of_mass = cmd.centerofmass("ligand")
        ligand_centers[ligand_id] = tuple(round(coord, 3) for coord in center_of_mass)
        cmd.delete("ligand")  # Clear the selection

    cmd.delete("all")  # Clear the loaded structure
    return ligand_centers


def calculate_ligand_centers(pdb_file):
    """
    Calculate the center of mass for each ligand in the structure using PyMOL.

    Args:
        pdb_file (str): Path to the PDB file.

    Returns:
        dict: Ligand identifiers as keys and their center of mass coordinates as values.
    """
    cmd.load(pdb_file, "protein")
    cmd.select("ligands", "organic")
    
    ligand_centers = {}
    stored.stored_atoms = []  # Corrected: Define `stored_atoms` in the PyMOL namespace

    # Iterate through ligands and collect unique identifiers
    cmd.iterate("ligands", "stored.stored_atoms.append((chain, resn, resi))")

    unique_ligands = set(stored.stored_atoms)  # Get unique ligand identifiers

    for chain, resn, resi in unique_ligands:
        ligand_id = f"{chain}_{resn}_{resi}"
        cmd.select("ligand", f"chain {chain} and resn {resn} and resi {resi}")
        center_of_mass = cmd.centerofmass("ligand")
        ligand_centers[ligand_id] = tuple(round(coord, 3) for coord in center_of_mass)
        cmd.delete("ligand")  # Clear selection

    cmd.delete("all")  # Clear everything from PyMOL session
    return ligand_centers

def calculate_ligand_diameter(pdb_file):
    """
    Calculate the diameter of each ligand (maximum pairwise distance between its atoms).

    Args:
        pdb_file (str): Path to the PDB file.

    Returns:
        dict: Ligand identifiers as keys and their diameters as values.
    """
    cmd.load(pdb_file, "protein")
    cmd.select("ligands", "organic")

    ligand_diameters = {}

    # Store ligand atom coordinates
    stored.ligand_atoms = []
    cmd.iterate_state(1, "ligands", "stored.ligand_atoms.append((chain, resn, resi, x, y, z))")

    unique_ligands = {}
    for chain, resn, resi, x, y, z in stored.ligand_atoms:
        ligand_id = f"{chain}_{resn}_{resi}"
        if ligand_id not in unique_ligands:
            unique_ligands[ligand_id] = []
        unique_ligands[ligand_id].append((x, y, z))

    # Compute ligand diameters
    for ligand_id, coordinates in unique_ligands.items():
        if len(coordinates) > 1:
            max_distance = max(
                np.linalg.norm(np.array(p1) - np.array(p2))
                for p1, p2 in combinations(coordinates, 2)
            )
            ligand_diameters[ligand_id] = round(max_distance, 3)
        else:
            ligand_diameters[ligand_id] = 0.0  # Single atom ligand, diameter is zero

    cmd.delete("all")  # Clear the loaded structure
    return ligand_diameters


def calculate_grid_size_ligand(ligand_diameter):
    """
    Compute the grid size based on the ligand diameter.

    Args:
        ligand_diameter (float): Diameter of the ligand.

    Returns:
        float: Computed grid size.
    """
    return round(16 + (0.8 * ligand_diameter), 3)


def define_bounding_box_ligand(ligand_centers, ligand_diameters):
    """
    Define a bounding box around the ligands based on their diameters.

    Args:
        ligand_centers (dict): Ligand identifiers and their center of mass coordinates.
        ligand_diameters (dict): Ligand identifiers and their diameters.

    Returns:
        dict: Bounding box coordinates for each ligand.
    """
    bounding_boxes = {}

    for ligand_id, center in ligand_centers.items():
        if ligand_id not in ligand_diameters:
            print(f"Warning: Missing diameter for ligand {ligand_id}")
            continue

        grid_size = calculate_grid_size_ligand(ligand_diameters[ligand_id])

        min_coords = [center[i] - grid_size / 2 for i in range(3)]
        max_coords = [center[i] + grid_size / 2 for i in range(3)]

        bounding_boxes[ligand_id] = {"min": min_coords, "max": max_coords}

        # Add pseudoatoms in PyMOL for visualization
        cmd.pseudoatom(f"box_{ligand_id}_min", pos=min_coords, color="blue")
        cmd.pseudoatom(f"box_{ligand_id}_max", pos=max_coords, color="red")

        print(f"Bounding box for {ligand_id}: Min={min_coords}, Max={max_coords}")

    return bounding_boxes

def select_cluster_define_bounding_box(df, cluster_id, cluster_column, selection_name="cluster_selection"):
    """
    Select residues based on the clustering method and create a bounding box in PyMOL.

    Args:
        df (pd.DataFrame): DataFrame containing residue data.
        cluster_id (int): The cluster ID to select residues for.
        cluster_column (str): Column name that stores cluster assignments.
        selection_name (str): Name of the selection in PyMOL.

    Returns:
        dict: Bounding box coordinates { "min": np.array, "max": np.array } for the selected cluster.
    """
    # Ensure required columns exist
    required_columns = {"chain", "resi", "center_of_mass_x", "center_of_mass_y", "center_of_mass_z"}
    missing_columns = required_columns - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing columns in DataFrame: {missing_columns}")

    # Filter residues based on the selected cluster
    selected_residues = df[df[cluster_column] == cluster_id]
    
    if selected_residues.empty:
        print(f"⚠️ No residues found for cluster {cluster_id} in column '{cluster_column}'")
        return None

    # Construct a PyMOL selection string
    selection_string = " or ".join(
        [f"(chain {row['chain']} and resi {row['resi']})" for _, row in selected_residues.iterrows()]
    )

    # Check if PyMOL is available
    try:
        cmd.select(selection_name, selection_string)
    except Exception as e:
        print(f"⚠️ PyMOL selection failed: {e}")
        return None

    # Compute the bounding box
    min_coords = selected_residues[['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].min().to_numpy()
    max_coords = selected_residues[['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].max().to_numpy()

    # Convert coordinates to float for compatibility
    min_coords = np.array([float(coord) for coord in min_coords])
    max_coords = np.array([float(coord) for coord in max_coords])

    # Add pseudoatoms for visualization
    try:
        cmd.pseudoatom(f"{selection_name}_box_min", pos=min_coords.tolist(), color="blue")
        cmd.pseudoatom(f"{selection_name}_box_max", pos=max_coords.tolist(), color="red")
    except Exception as e:
        print(f"⚠️ Failed to create pseudoatoms in PyMOL: {e}")

    print(f"✅ Selection '{selection_name}' created for cluster {cluster_id}")
    print(f"📦 Bounding box: Min={min_coords.tolist()}, Max={max_coords.tolist()}")
    
    #euclidean distance
    size = np.linalg.norm(max_coords - min_coords)

    return {"min": min_coords, "max": max_coords, "size": size}


def calculate_dice_score(cluster_selection, ligand_selection):
    """
    Compute the Dice similarity coefficient between a cluster and a ligand.

    Args:
        cluster_selection (str): PyMOL selection name for the cluster.
        ligand_selection (str): PyMOL selection name for the ligand.

    Returns:
        float: Dice score indicating spatial overlap (higher means better overlap).
    """
    # Get atoms in selections
    cluster_atoms = set(cmd.index(cluster_selection))
    ligand_atoms = set(cmd.index(ligand_selection))

    if not cluster_atoms or not ligand_atoms:
        print(f"Error: One or both selections are empty: {cluster_selection}, {ligand_selection}")
        return 0.0

    # Compute Dice coefficient
    intersection = len(cluster_atoms & ligand_atoms)
    dice_score = (2 * intersection) / (len(cluster_atoms) + len(ligand_atoms))

    print(f"Dice Score between {cluster_selection} and {ligand_selection}: {round(dice_score, 3)}")
    
    return round(dice_score, 3)

def calculate_distances_between_clusters_and_ligands(cluster_centers, ligand_centers):
    """
    Calculate the minimum distance between each cluster and the ligands.

    Args:
        cluster_centers (dict): Cluster IDs and their center of mass coordinates.
        ligand_centers (dict): Ligand identifiers and their center of mass coordinates.

    Returns:
        dict: Cluster IDs as keys and tuples of (closest ligand ID, distance) as values.
    """
    distances = {}

    for cluster_id, cluster_center in cluster_centers.items():
        min_distance = float('inf')
        closest_ligand = None

        for ligand_id, ligand_center in ligand_centers.items():
            distance = np.linalg.norm(np.array(cluster_center) - np.array(ligand_center))
            if distance < min_distance:
                min_distance = round(distance, 3)
                closest_ligand = ligand_id

        distances[cluster_id] = (closest_ligand, min_distance)

    return distances

def evaluate_clustering(pdb_file, df, cluster_column):
    """
    Evaluate the clustering by calculating distances between cluster centers and ligand centers.

    Args:
        pdb_file (str): Path to the PDB file.
        df (pd.DataFrame): DataFrame containing residue and cluster information.
        cluster_column (str): Column name containing cluster IDs.

    Returns:
        dict: Distances between each cluster and the closest ligand.
    """
    print("Calculating cluster centers...")
    cluster_centers = calculate_cluster_centers(df, cluster_column, pdb_file)

    print("Calculating ligand centers...")
    ligand_centers = calculate_ligand_centers(pdb_file)

    print("Calculating distances between clusters and ligands...")
    distances = calculate_distances_between_clusters_and_ligands(cluster_centers, ligand_centers)

    for cluster_id, (ligand_id, distance) in distances.items():
        print(f"Cluster {cluster_id} -> Closest Ligand: {ligand_id}, Distance: {distance} Å")

    return distances

# Pipeline Function
def run_pipeline(data_path, pdb_file, clustering_method, cluster_params, pbind_column='p(bind)', cutoff_method='median_std', std_factor=0.5):
    """
    Run the clustering pipeline, including clustering, normalization, filtering, and evaluation.

    Args:
        data_path (str): Path to the data directory.
        pdb_file (str): Path to the PDB file.
        clustering_method (str): Clustering method to use ('kmeans' or 'spectral').
        cluster_params (dict): Parameters for the clustering method.
        pbind_column (str): Column name for binding probabilities.
        cutoff_method (str): Method for calculating cutoff ('median_std' or 'percentile').
        std_factor (float): Factor for standard deviation when calculating cutoff.

    Returns:
        dict: Evaluation results including distances between clusters and ligands.
    """
    print("Loading data...")
    combined_df = process_all_pdb_files(data_path)

    print("Applying clustering...")
    if clustering_method == 'kmeans':
        n_clusters = cluster_params.get('n_clusters', 3)
        combined_df, _ = kmeans_clustering(combined_df, feature_columns=cluster_params['feature_columns'], n_clusters=n_clusters)
    elif clustering_method == 'spectral':
        n_clusters = cluster_params.get('n_clusters', 3)
        affinity = cluster_params.get('affinity', 'nearest_neighbors')
        combined_df, _ = spectral_clustering(combined_df, feature_columns=cluster_params['feature_columns'], n_clusters=n_clusters, affinity=affinity)
    else:
        raise ValueError("Invalid clustering method. Choose 'kmeans' or 'spectral'.")

    print("Normalizing p(bind) within clusters...")
    combined_df = normalize_within_clusters(combined_df, cluster_column=clustering_method+'_cluster', pbind_column=pbind_column)

    print("Filtering residues based on p(bind) cutoff...")
    filtered_df = filter_by_pbind_cutoff(combined_df, pbind_column=pbind_column, cluster_column=clustering_method+'_cluster', cutoff_method=cutoff_method, std_factor=std_factor)

    print("Evaluating clustering...")
    evaluation_results = evaluate_clustering(pdb_file, filtered_df, cluster_column=clustering_method+'_cluster')

    return {
        'clustered_df': combined_df,
        'filtered_df': filtered_df,
        'evaluation_results': evaluation_results
    }


In [4]:
# Execute the data processing pipeline

# Process all files and get the combined DataFrame
combined_df = process_all_pdb_files(data_path)

# Ensure 'PDBcode' is present in combined_df
if 'PDBcode' not in combined_df.columns:
    raise ValueError("Error: 'PDBcode' column is missing in combined_df. Check preprocessing.")

# Create a dictionary of dataframes grouped by PDBcode
dfs = {pdb: df for pdb, df in combined_df.groupby('PDBcode')}

# Check if extracted dataframes match the original CSV files
for pdb in combined_df['PDBcode'].unique():
    original_csv_path = os.path.join(data_path, f"results_{pdb}.csv")
    
    if not os.path.exists(original_csv_path):
        print(f"Warning: Original CSV file missing for {pdb}. Skipping...")
        continue
    
    original_df = pd.read_csv(original_csv_path)
    
    if pdb in dfs:
        extracted_df = dfs[pdb]
    else:
        print(f"Warning: {pdb} missing in processed data. Reprocessing...")
        extracted_df = process_and_update_pdb_data(data_path, pdb)

    # Check if the number of entries match
    if len(original_df) == len(extracted_df):
        print(f"{pdb}: ✅ Match - {len(original_df)} entries")
    else:
        print(f"{pdb}: ❌ Mismatch - Original: {len(original_df)} entries, Extracted: {len(extracted_df)} entries")
        
        # Attempt to fix by loading updated version
        updated_csv_path = os.path.join(data_path, f"results_{pdb}_updated.csv")
        
        if os.path.exists(updated_csv_path):
            print(f"Fixing process by using updated data for {pdb}...")
            extracted_df = pd.read_csv(updated_csv_path)
            dfs[pdb] = extracted_df  # Update dictionary
            
            if len(original_df) == len(extracted_df):
                print(f"{pdb}: ✅ Fixed - Match after update: {len(original_df)} entries")
            else:
                print(f"{pdb}: ❌ Still Mismatch after update: {len(original_df)} vs {len(extracted_df)} entries")
        else:
            print(f"Error: No updated file found for {pdb}. Manual check required.")

# Define feature and p(bind) column
feature_columns = ['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']
pbind_column = 'p(bind)'

# Process features for each PDB entry
for pdb_code in dfs.keys():
    # Normalize spatial features
    dfs[pdb_code], feature_scalers = normalize_features(dfs[pdb_code], feature_columns)
    
    # Apply multiple p(bind) weight factors
    dfs[pdb_code], pbind_scalers = weighted_pbind(dfs[pdb_code], pbind_column, weights=[1, 2, 5, 10, 20, 50, 100])

    # Process protein metadata (diameter, center of mass)
    dfs[pdb_code] = process_protein_data(pdb_code, dfs, data_path)

    # Rename and clean up dataframe
    dfs[pdb_code].rename(columns=lambda x: x.strip(), inplace=True)
    dfs[pdb_code].drop(columns="Unnamed: 0", inplace=True, errors="ignore")  # Avoid errors if column missing

# Print an example output for verification
print("\n✅ Processed Data Example:")
print(dfs[list(dfs.keys())[0]].head())  # Print first PDB's processed dataframe


1qcf: ✅ Match - 450 entries
1ubq: ✅ Match - 76 entries
3g5d: ✅ Match - 513 entries
3zln: ✅ Match - 144 entries
4f9w: ✅ Match - 336 entries
1lyz: ✅ Match - 129 entries
1hvy: ✅ Match - 1152 entries
3hvc: ✅ Match - 327 entries
3cpa: ✅ Match - 307 entries
1pw6: ✅ Match - 250 entries
1ema: ✅ Match - 225 entries
1kv1: ✅ Match - 331 entries
1be9: ✅ Match - 120 entries
3ptb: ✅ Match - 220 entries
6o0k: ✅ Match - 141 entries
2bal: ✅ Match - 338 entries
4pti: ✅ Match - 58 entries
1yer: ✅ Match - 207 entries
1cz2: ✅ Match - 90 entries
3qkd: ✅ Match - 282 entries
1ao6: ✅ Match - 1156 entries
1h61: ✅ Match - 364 entries
1ny3: ✅ Match - 277 entries

✅ Processed Data Example:
     chain  resi resn   p(bind)  \
8133     A     5    S  0.008225   
8134     A     6    E  0.013932   
8135     A     7    V  0.044013   
8136     A     8    A  0.005330   
8137     A     9    H  0.007117   

                                       resn_coordinates  center_of_mass_x  \
8133  [('N', 56.653, 51.017, 34.141), ('C'

In [5]:
# Median clustering function

import re


def silhouette_analysis(data, max_clusters=10):
    """
    Perform silhouette analysis for a range of clusters and suggest the optimal n_clusters.

    Args:
        data (array-like): Dataset to cluster.
        max_clusters (int): Maximum number of clusters to evaluate.

    Returns:
        tuple: Suggested n_clusters, silhouette scores for each k.
    """
    silhouette_scores = []
    
    for n_clusters in range(2, max_clusters + 1):  # Silhouette starts from k=2
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(data)
        silhouette_scores.append(silhouette_score(data, labels))

    suggested_k = np.argmax(silhouette_scores) + 2  # +2 because we start at k=2
    print(f"Suggested number of clusters (Silhouette): {suggested_k}")
    print("Silhouette Scores:", silhouette_scores)
    
    return suggested_k, silhouette_scores


def kmeans_clustering(df, feature_columns, n_clusters):
    """
    Perform KMeans clustering and add cluster labels to the DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame.
        feature_columns (list): List of feature column names.
        n_clusters (int): Number of clusters.

    Returns:
        pd.DataFrame: Updated DataFrame with cluster labels.
        KMeans: Fitted KMeans model.
    """
    features = df[feature_columns].values
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df['kmeans_cluster'] = kmeans.fit_predict(features)
    return df, kmeans


def normalize_within_clusters(df, cluster_column, pbind_column):
    """
    Normalize p(bind) values within each cluster.

    Args:
        df (pd.DataFrame): Input DataFrame.
        cluster_column (str): Column containing cluster IDs.
        pbind_column (str): Column containing p(bind) values.

    Returns:
        pd.DataFrame: Updated DataFrame with normalized p(bind) values.
    """
    df[f"normalized_{pbind_column}"] = df.groupby(cluster_column)[pbind_column].transform(
        lambda x: (x - x.min()) / (x.max() - x.min())
    )
    return df


def filter_by_pbind_cutoff(df, pbind_column, cluster_column, cutoff_method, std_factor):
    """
    Filter residues based on p(bind) cutoff.

    Args:
        df (pd.DataFrame): Input DataFrame.
        pbind_column (str): Column containing p(bind) values.
        cluster_column (str): Column containing cluster IDs.
        cutoff_method (str): Method for calculating cutoff ('median_std' or 'percentile').
        std_factor (float): Factor for standard deviation in cutoff calculation.

    Returns:
        pd.DataFrame: Filtered DataFrame.
    """
    filtered_df = pd.DataFrame()

    for cluster_id in df[cluster_column].unique():
        cluster_data = df[df[cluster_column] == cluster_id]
        if cutoff_method == 'median_std':
            median_value = cluster_data[pbind_column].median()
            std_value = cluster_data[pbind_column].std()
            cutoff = median_value + std_factor * std_value
        elif cutoff_method == 'percentile':
            cutoff = cluster_data[pbind_column].quantile(0.95)
        else:
            raise ValueError("Invalid cutoff method. Use 'median_std' or 'percentile'.")

        filtered_cluster = cluster_data[cluster_data[pbind_column] > cutoff]
        filtered_df = pd.concat([filtered_df, filtered_cluster])

    return filtered_df.reset_index(drop=True)


def compute_centroids(df, cluster_column):
                # Compute the cluster centers and round to 3 decimal places
                centroids = df.groupby(cluster_column)[['p(bind)', 'center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].mean().round(3)
                centroids.sort_values(by='p(bind)')
                
                return centroids


def med_clustering(df, std_factors, feature_columns, pbind_column, pbind_weight):
    """ Perform clustering to identify residues with high binding probabilities. """

    
    # 🔹 Step 1:Compute median and std for filtering
    median_value = df[pbind_column].median()
    std_value = df[pbind_column].std()
    
    cutoff = median_value + std_factors * std_value
    df_filtered = df.loc[df[pbind_column] > cutoff]
            
    feature_columns = feature_columns + [pbind_column]*pbind_weight
    
    # 🔹 Step 2: Determine optimal number of clusters safely
    optimal_kmeans, _ = silhouette_analysis(df_filtered[feature_columns].values, max_clusters=min(10, len(df_filtered) - 1))
    
    # 🔹 Step 3: Run Clustering with Optimal Clusters
    df_kmeans, _ = kmeans_clustering(df_filtered, feature_columns, n_clusters=optimal_kmeans)
    
    # Store cluster labels
    #df_filtered[f'kmeans_pbind'] = df_kmeans['kmeans_cluster']
    
    # 🔹 Step 4: Compute Cluster Center
    centroids = compute_centroids(df_kmeans, 'kmeans_cluster')
    
    # Compute residue IDs for each cluster: pls 
    df_kmeans['residue_id'] = df_kmeans['chain'] + '_' + df_kmeans['resi'].astype(str)
    centroids["residue_ids"] = df_kmeans.groupby('kmeans_cluster')['residue_id'].apply(list)
    
    # make in this format w/o , and []: A_105 A_106 A_107
    centroids["residue_ids"] = centroids["residue_ids"].apply(lambda x: ' '.join(x))
    
    # Rename columns
    centroids['rank'] = centroids['p(bind)'].rank(ascending=False).astype(int)
    centroids['name'] = "pocket" + centroids['rank'].astype(str)
    centroids['name'] = centroids['name'].apply(lambda x: re.sub(r'\.0', '', x))
    
    # 🔹 Rename columns
    centroids['rank'] = centroids['p(bind)'].rank(ascending=False, method='dense').astype(int)
    centroids['name'] = "pocket" + centroids['rank'].astype(str)
    centroids['name'] = centroids['name'].apply(lambda x: re.sub(r'\.0', '', x))
        
    # Select and rename columns
    centroids = centroids[['name', 'rank', 'p(bind)', 'center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z', 'residue_ids']]
    centroids.rename(columns={'p(bind)': 'probability', 'center_of_mass_x': 'center_x', 'center_of_mass_y': 'center_y', 'center_of_mass_z': 'center_z'}, inplace=True)
        
    # Sort by binding probability
    centroids.sort_values(by='probability', ascending=False, inplace=True)
        
    # Save to CSV
    centroids.to_csv("pockets.csv", index=False)

    # 🔹 Prepare the residues DataFrame
    residues_df = df[['chain', 'resi','resn', 'p(bind)', 'center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].copy()
    residues_df['pocket'] = np.nan  # Initialize pocket column
        
    # Assign pocket values based on clustering results
    cluster_mapping = dict(zip(df_kmeans['residue_id'], df_kmeans['kmeans_cluster'] + 1))  # Map cluster IDs to pockets (1-based index)
    residues_df['residue_id'] = residues_df['chain'] + '_' + residues_df['resi'].astype(str)
    residues_df['pocket'] = residues_df['residue_id'].map(cluster_mapping).fillna(0).astype(int)  # NA means no cluster -> assign 0
        
    # Save final results
    residues_df.to_csv("residues.csv", index=False)

    return residues_df, centroids 

df = dfs['3hvc']
pdb_file =f"{data_path}/3hvc.pdb"
ligand_center = calculate_ligand_centers(pdb_file)
feature_columns = ['center_of_mass_x_normalized', 'center_of_mass_y_normalized', 'center_of_mass_z_normalized']
pbind_column = 'p(bind)_weight_1'


df_kmean, centroid = med_clustering(df, 2.5, feature_columns, pbind_column, 100)

    
    

Suggested number of clusters (Silhouette): 2
Silhouette Scores: [0.6355500000273376, 0.5408567437236194, 0.47825751533187444, 0.5465603793327393, 0.5558734824532885, 0.4738380218244981, 0.3953970140689803, 0.30661873196590006, 0.2537951909943017]


/var/folders/v4/rd2_123918b6cj4v96gtlmkh0000gn/T/ipykernel_62066/3448447106.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kmeans_cluster'] = kmeans.fit_predict(features)
/var/folders/v4/rd2_123918b6cj4v96gtlmkh0000gn/T/ipykernel_62066/3448447106.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kmeans['residue_id'] = df_kmeans['chain'] + '_' + df_kmeans['resi'].astype(str)


In [6]:
df_kmean.pocket.value_counts()

pocket
0    311
2      9
1      7
Name: count, dtype: int64

In [7]:
centroid

,name,rank,probability,center_x,center_y,center_z,residue_ids
kmeans_cluster,,,,,,,
1,pocket1,1,0.840,-3.750,35.536,20.913,A_36 A_38 A_51 A_53 A_71 A_84 A_106 A_167 A_168
0,pocket2,2,0.577,0.726,32.606,20.853,A_31 A_67 A_104 A_109 A_149 A_150 A_155


In [8]:
df_kmean

,chain,resi,resn,p(bind),center_of_mass_x,center_of_mass_y,center_of_mass_z,pocket,residue_id
2800,A,5,R,0.039529,-24.874,37.998,29.439,0,A_5
2801,A,6,P,0.034048,-23.549,40.555,27.539,0,A_6
2802,A,7,T,0.003536,-26.034,40.724,23.731,0,A_7
2803,A,8,F,0.006006,-23.113,37.045,23.328,0,A_8
2804,A,9,Y,0.001696,-22.688,38.464,17.061,0,A_9
...,...,...,...,...,...,...,...,...,...
3122,A,348,F,0.029051,-11.070,40.469,33.250,0,A_348
3123,A,349,V,0.026582,-10.757,43.710,35.590,0,A_349
3124,A,350,P,0.035559,-11.266,45.571,32.610,0,A_350
3125,A,351,P,0.034058,-8.268,47.652,31.057,0,A_351


In [10]:
df_kmean.shape

(327, 9)

In [11]:
residues_file = "/Users/nicha/dev/p2rank/distro/test_output/predict_3hvc_stripped/3hvc_stripped.pdb_residues.csv"
# rename the column name remove all the space
residue_df = pd.read_csv(residues_file)
residue_df

,chain,residue_label,residue_name,score,zscore,probability,pocket
0,A,5,ARG,0.3160,-0.2033,0.0153,0
1,A,6,PRO,0.3657,-0.1756,0.0200,5
2,A,7,THR,0.1179,-0.3138,0.0030,0
3,A,8,PHE,0.2493,-0.2405,0.0101,0
4,A,9,TYR,0.0882,-0.3303,0.0020,0
...,...,...,...,...,...,...,...
322,A,348,PHE,0.1533,-0.2940,0.0045,5
323,A,349,VAL,0.1597,-0.2904,0.0048,0
324,A,350,PRO,0.1545,-0.2934,0.0045,0
325,A,351,PRO,0.2739,-0.2268,0.0119,4


In [12]:
# Compute ligand centers and diameters
ligand_centers = calculate_ligand_centers(pdb_file)
ligand_diameters = calculate_ligand_diameter(pdb_file)

# Compute grid sizes
grid_sizes = {lig_id: calculate_grid_size_ligand(diameter) for lig_id, diameter in ligand_diameters.items()}

# Print results
print("Ligand Centers:", ligand_centers)
print("Ligand Diameters:", ligand_diameters)
print("Computed Grid Sizes:", grid_sizes)

Ligand Centers: {'A_GG5_361': (-3.732, 37.54, 19.149), 'A_GG5_362': (24.619, 22.289, 26.946)}
Ligand Diameters: {'A_GG5_361': 8.652, 'A_GG5_362': 8.632}
Computed Grid Sizes: {'A_GG5_361': 22.922, 'A_GG5_362': 22.906}


In [13]:
define_bounding_box_ligand(ligand_centers, ligand_diameters)

Bounding box for A_GG5_361: Min=[-15.193000000000001, 26.079, 7.688000000000001], Max=[7.729, 49.001, 30.61]
Bounding box for A_GG5_362: Min=[13.166, 10.836000000000002, 15.493000000000002], Max=[36.072, 33.742000000000004, 38.399]


{'A_GG5_361': {'min': [-15.193000000000001, 26.079, 7.688000000000001],
  'max': [7.729, 49.001, 30.61]},
 'A_GG5_362': {'min': [13.166, 10.836000000000002, 15.493000000000002],
  'max': [36.072, 33.742000000000004, 38.399]}}

In [14]:
df_kmean

,chain,resi,resn,p(bind),center_of_mass_x,center_of_mass_y,center_of_mass_z,pocket,residue_id
2800,A,5,R,0.039529,-24.874,37.998,29.439,0,A_5
2801,A,6,P,0.034048,-23.549,40.555,27.539,0,A_6
2802,A,7,T,0.003536,-26.034,40.724,23.731,0,A_7
2803,A,8,F,0.006006,-23.113,37.045,23.328,0,A_8
2804,A,9,Y,0.001696,-22.688,38.464,17.061,0,A_9
...,...,...,...,...,...,...,...,...,...
3122,A,348,F,0.029051,-11.070,40.469,33.250,0,A_348
3123,A,349,V,0.026582,-10.757,43.710,35.590,0,A_349
3124,A,350,P,0.035559,-11.266,45.571,32.610,0,A_350
3125,A,351,P,0.034058,-8.268,47.652,31.057,0,A_351


In [31]:
def process_residue_file(residue_file, pdb_file):
    """
    Reads a PDB file and updates a DataFrame with residue center of mass.

    Args:
        residue_file (str): Path to the residue CSV file.
        pdb_file (str): Path to the PDB file.

    Returns:
        pd.DataFrame: Updated DataFrame with center of mass information.
    """
    residues_df = pd.read_csv(residue_file)
    
    #remove the white space in the column name
    residues_df.rename(columns=lambda x: x.strip(), inplace=True)
    #print(residues_df.columns)
    
    # Rename columns residue_label to resi
    residues_df.rename(columns={'residue_label':'resi'}, inplace=True)
    #print(residues_df.columns)
    
    
    for index, row in residues_df.iterrows():
        residue_number = row["resi"]
        chain_id = row["chain"].strip()
        coordinates = extract_residue_coordinates(pdb_file, residue_number, chain_id)
        if coordinates:
            center_of_mass = calculate_weighted_center_of_mass(coordinates)
            residues_df.loc[index, "resn_coordinates"] = str(coordinates)
            residues_df.loc[index, ["center_of_mass_x", "center_of_mass_y", "center_of_mass_z"]] = center_of_mass
        else:
            print(f"Residue {residue_number} in chain {chain_id} not found in {pdb_file}.")

    return residues_df

# grid box preidction from p2rank

def calculate_size_residues_df(residue_df, pocket_id):
    """
    Calculates the size of a binding pocket based on residue coordinates.

    Args:
        residue_df (pd.DataFrame): DataFrame containing residue information.
        pocket_id (int): ID of the pocket to analyze.

    Returns:
        dict: Dictionary containing pocket size and bounding box dimensions.
    """

    pocket_id = 1
    selected_residues = residue_df.loc[residue_df["pocket"] == pocket_id]
    selection_name="cluster_selection"

    if selected_residues.empty:
            print(f"⚠️ No residues found for cluster {pocket_id} in column 'pocket'")
            return None
        

    # Construct a PyMOL selection string
    selection_string = " or ".join(
            [f"(chain {row['chain']} and resi {row['resi']})" for _, row in selected_residues.iterrows()]
        )#

    # Check if PyMOL is available
    try:
        cmd.select(selection_name, selection_string)
    except Exception as e:
        print(f"⚠️ PyMOL selection failed: {e}")
        return None

    # Compute the bounding box
    min_coords = selected_residues[['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].min().to_numpy()
    max_coords = selected_residues[['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].max().to_numpy()

    # Convert coordinates to float for compatibility
    min_coords = np.array([float(coord) for coord in min_coords])
    max_coords = np.array([float(coord) for coord in max_coords])

    # Add pseudoatoms for visualization
    try:
        cmd.pseudoatom(f"{selection_name}_box_min", pos=min_coords.tolist(), color="blue")
        cmd.pseudoatom(f"{selection_name}_box_max", pos=max_coords.tolist(), color="red")
    except Exception as e:
        print(f"⚠️ Failed to create pseudoatoms in PyMOL: {e}")
        
    #print(f"📦 Bounding box: Min={min_coords.tolist()}, Max={max_coords.tolist()}")
    
    #euclidean distance
    size = round(np.linalg.norm(max_coords - min_coords),3)
    
    return {"min": min_coords, "max": max_coords, "size": size}

def calculate_pocket_data(residues_file, pocket_file, pdb_file, pocket_id):
    """
    Processes residue and pocket data, calculates the binding pocket size, 
    and extracts its center coordinates.

    Args:
        residues_file (str): Path to the residues CSV file.
        pocket_file (str): Path to the pocket predictions CSV file.
        pdb_file (str): Path to the PDB file.
        pocket_id (int): ID of the pocket to analyze.

    Returns:
        dict: Contains the center coordinates and size of the pocket.
    """

    # Load and clean residues data
    residue_df = pd.read_csv(residues_file)
    residue_df.rename(columns=lambda x: x.strip(), inplace=True)
    residue_df = process_residue_file(residues_file, pdb_file)

    # Load and clean pocket data
    pocket_df = pd.read_csv(pocket_file)
    pocket_df.rename(columns=lambda x: x.strip(), inplace=True)

    # Compute binding pocket size
    info_binding_pocket = calculate_size_residues_df(residue_df, pocket_id)

    # Select pocket-specific residues
    selected_pocket = residue_df.loc[residue_df['pocket'] == pocket_id]
    
    selected_pocket.columns

    # Extract center coordinates and size
    coordinate = selected_pocket[['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].values[0]
    size = info_binding_pocket['size']

    return {
        "coordinate": coordinate,
        "size": round(size, 3)
    }
    
import os

def create_config_autodock(residues_file, pocket_file, pdb_file, pocket_id, output_dir):
    """
    Create an AutoDock configuration file based on pocket data.

    Args:
        residues_file (str): Path to residues CSV file.
        pocket_file (str): Path to pocket file.
        pdb_file (str): Path to protein PDB file.
        pocket_id (int): ID of the binding pocket.
        output_dir (str): Directory to save the config file.

    Returns:
        str: Path to the generated config file.
    """

    # Extract protein name from PDB file
    protein_name = os.path.splitext(os.path.basename(pdb_file))[0]

    # Calculate pocket data
    result = calculate_pocket_data(residues_file, pocket_file, pdb_file, pocket_id)

    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define config file path
    config_file = os.path.join(output_dir, f"config_{protein_name}_pocket_{pocket_id}.txt")

    # Write the configuration file
    with open(config_file, "w") as f:
        f.write(f"center_x = {result['coordinate'][0]}\n")
        f.write(f"center_y = {result['coordinate'][1]}\n")
        f.write(f"center_z = {result['coordinate'][2]}\n")
        f.write(f"size_x = {result['size']}\n")
        f.write(f"size_y = {result['size']}\n")
        f.write(f"size_z = {result['size']}\n")
        f.write("exhaustiveness = 8\n")  # Example extra parameter
        f.write("num_modes = 20\n")  # Example docking modes

    print(f"✅ Config file created: {config_file}")
    
    return config_file

 
def create_config_ligand_based(pdb_file, output_dir):   
    # Compute ligand centers and diameters
    ligand_centers = calculate_ligand_centers(pdb_file)
    ligand_diameters = calculate_ligand_diameter(pdb_file)

    # Compute grid sizes
    grid_sizes = {lig_id: calculate_grid_size_ligand(diameter) for lig_id, diameter in ligand_diameters.items()}
    
     # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define config file path
    config_file = os.path.join(output_dir, f"config_{protein_name}_pocket_{pocket_id}.txt")

    # Write the configuration file
    with open(config_file, "w") as f:
        f.write(f"center_x = {result['coordinate'][0]}\n")
        f.write(f"center_y = {result['coordinate'][1]}\n")
        f.write(f"center_z = {result['coordinate'][2]}\n")
        f.write(f"size_x = {result['size']}\n")
        f.write(f"size_y = {result['size']}\n")
        f.write(f"size_z = {result['size']}\n")
        f.write("exhaustiveness = 8\n")  # Example extra parameter
        f.write("num_modes = 20\n")  # Example docking modes

    print(f"✅ Config file created: {config_file}")
    
    return config_file
    
    
def process_residue_file(residue_file, pdb_file):
    """
    Reads a PDB file and updates a DataFrame with residue center of mass.

    Args:
        residue_file (str): Path to the residue CSV file.
        pdb_file (str): Path to the PDB file.

    Returns:
        pd.DataFrame: Updated DataFrame with center of mass information.
    """
    residues_df = pd.read_csv(residue_file)
    
    #remove the white space in the column name
    residues_df.rename(columns=lambda x: x.strip(), inplace=True)
    #print(residues_df.columns)
    
    # Rename columns residue_label to resi
    residues_df.rename(columns={'residue_label':'resi'}, inplace=True)
    #print(residues_df.columns)
    
    
    for index, row in residues_df.iterrows():
        residue_number = row["resi"]
        chain_id = row["chain"].strip()
        coordinates = extract_residue_coordinates(pdb_file, residue_number, chain_id)
        if coordinates:
            center_of_mass = calculate_weighted_center_of_mass(coordinates)
            residues_df.loc[index, "resn_coordinates"] = str(coordinates)
            residues_df.loc[index, ["center_of_mass_x", "center_of_mass_y", "center_of_mass_z"]] = center_of_mass
        else:
            print(f"Residue {residue_number} in chain {chain_id} not found in {pdb_file}.")

    return residues_df

# grid box preidction from p2rank

def calculate_size_residues_df(residue_df, pocket_id):
    """
    Calculates the size of a binding pocket based on residue coordinates.

    Args:
        residue_df (pd.DataFrame): DataFrame containing residue information.
        pocket_id (int): ID of the pocket to analyze.

    Returns:
        dict: Dictionary containing pocket size and bounding box dimensions.
    """

    pocket_id = 1
    selected_residues = residue_df.loc[residue_df["pocket"] == pocket_id]
    selection_name="cluster_selection"

    if selected_residues.empty:
            print(f"⚠️ No residues found for cluster {pocket_id} in column 'pocket'")
            return None
        

    # Construct a PyMOL selection string
    selection_string = " or ".join(
            [f"(chain {row['chain']} and resi {row['resi']})" for _, row in selected_residues.iterrows()]
        )#

    # Check if PyMOL is available
    try:
        cmd.select(selection_name, selection_string)
    except Exception as e:
        print(f"⚠️ PyMOL selection failed: {e}")
        return None

    # Compute the bounding box
    min_coords = selected_residues[['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].min().to_numpy()
    max_coords = selected_residues[['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].max().to_numpy()

    # Convert coordinates to float for compatibility
    min_coords = np.array([float(coord) for coord in min_coords])
    max_coords = np.array([float(coord) for coord in max_coords])

    # Add pseudoatoms for visualization
    try:
        cmd.pseudoatom(f"{selection_name}_box_min", pos=min_coords.tolist(), color="blue")
        cmd.pseudoatom(f"{selection_name}_box_max", pos=max_coords.tolist(), color="red")
    except Exception as e:
        print(f"⚠️ Failed to create pseudoatoms in PyMOL: {e}")
        
    #print(f"📦 Bounding box: Min={min_coords.tolist()}, Max={max_coords.tolist()}")
    
    #euclidean distance
    size = round(np.linalg.norm(max_coords - min_coords),3)
    
    return {"min": min_coords, "max": max_coords, "size": size}

def calculate_pocket_data(residues_file, pocket_file, pdb_file, pocket_id):
    """
    Processes residue and pocket data, calculates the binding pocket size, 
    and extracts its center coordinates.

    Args:
        residues_file (str): Path to the residues CSV file.
        pocket_file (str): Path to the pocket predictions CSV file.
        pdb_file (str): Path to the PDB file.
        pocket_id (int): ID of the pocket to analyze.

    Returns:
        dict: Contains the center coordinates and size of the pocket.
    """

    # Load and clean residues data
    residue_df = pd.read_csv(residues_file)
    residue_df.rename(columns=lambda x: x.strip(), inplace=True)
    residue_df = process_residue_file(residues_file, pdb_file)

    # Load and clean pocket data
    pocket_df = pd.read_csv(pocket_file)
    pocket_df.rename(columns=lambda x: x.strip(), inplace=True)

    # Compute binding pocket size
    info_binding_pocket = calculate_size_residues_df(residue_df, pocket_id)

    # Select pocket-specific residues
    selected_pocket = residue_df.loc[residue_df['pocket'] == pocket_id]
    
    selected_pocket.columns

    # Extract center coordinates and size
    coordinate = selected_pocket[['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z']].values[0]
    size = info_binding_pocket['size']

    return {
        "coordinate": coordinate,
        "size": round(size, 3)
    }


def create_config_pocket_based(residues_file, pocket_file, pdb_file, pocket_id, output_dir):
    """
    Create an AutoDock configuration file based on pocket data.

    Args:
        residues_file (str): Path to residues CSV file.
        pocket_file (str): Path to pocket file.
        pdb_file (str): Path to protein PDB file.
        pocket_id (int): ID of the binding pocket.
        output_dir (str): Directory to save the config file.

    Returns:
        str: Path to the generated config file.
    """

    # Extract protein name from PDB file
    protein_name = os.path.splitext(os.path.basename(pdb_file))[0]

    # Calculate pocket data
    result = calculate_pocket_data(residues_file, pocket_file, pdb_file, pocket_id)

    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define config file path
    config_file = os.path.join(output_dir, f"config_{protein_name}_pocket_{pocket_id}.txt")

    # Write the configuration file
    with open(config_file, "w") as f:
        f.write(f"center_x = {result['coordinate'][0]}\n")
        f.write(f"center_y = {result['coordinate'][1]}\n")
        f.write(f"center_z = {result['coordinate'][2]}\n")
        f.write(f"size_x = {result['size']}\n")
        f.write(f"size_y = {result['size']}\n")
        f.write(f"size_z = {result['size']}\n")
        f.write("exhaustiveness = 8\n")  # Example extra parameter
        f.write("num_modes = 20\n")  # Example docking modes

    print(f"✅ Config file created: {config_file}")
    
    return config_file

 
def create_config_ligand_based(pdb_file, output_dir):
    """
    Create an AutoDock configuration file based on ligand data.

    Args:
        pdb_file (str): Path to the protein PDB file.
        output_dir (str): Directory to save the config file.

    Returns:
        dict: Paths to the generated config files.
    """
    # Extract protein name from PDB file
    protein_name = os.path.splitext(os.path.basename(pdb_file))[0]

    # Compute ligand centers and diameters
    ligand_centers = calculate_ligand_centers(pdb_file)
    ligand_diameters = calculate_ligand_diameter(pdb_file)

    # Compute grid sizes
    grid_sizes = {lig_id: calculate_grid_size_ligand(diameter) for lig_id, diameter in ligand_diameters.items()}

    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config_files = {}

    for lig_id, ligand_center in ligand_centers.items():
        grid_size = grid_sizes[lig_id]
        # Define config file path
        config_file = os.path.join(output_dir, f"config_{protein_name}_ligand_{lig_id}.txt")
        with open(config_file, "w") as f:
            f.write(f"center_x = {ligand_center[0]}\n")
            f.write(f"center_y = {ligand_center[1]}\n")
            f.write(f"center_z = {ligand_center[2]}\n")
            f.write(f"size_x = {grid_size}\n")
            f.write(f"size_y = {grid_size}\n")
            f.write(f"size_z = {grid_size}\n")
            f.write("exhaustiveness = 8\n")
            f.write("num_modes = 20\n")
        
        config_files[lig_id] = config_file

    print(f"✅ Config files created: {config_files}")
    
    return config_files
    
    
   
    

In [ ]:
# Create config file from the prank output
residues_file = "/Users/nicha/dev/p2rank/distro/test_output/predict_3hvc_stripped/3hvc_stripped.pdb_residues.csv"
pocket_file = "/Users/nicha/dev/p2rank/distro/test_output/predict_3hvc_stripped/3hvc_stripped.pdb_predictions.csv"
pdb_file = data_path + "/3hvc.pdb"
pocket_id = 1

result = calculate_pocket_data(residues_file, pocket_file, pdb_file, pocket_id)

create_config_pocket_based(residues_file, pocket_file, pdb_file, pocket_id, "/Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc")


✅ Config file created: /Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc/config_3hvc_pocket_1.txt


'/Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc/config_3hvc_pocket_1.txt'

In [ ]:
# create a config file from median clustering
residues_file_median = "/Users/nicha/dev/Protein-preparation-pipeline/notebooks/residues.csv"
pocket_file_median = "/Users/nicha/dev/Protein-preparation-pipeline/notebooks/pockets.csv"

pocket_id = 1

create_config_pocket_based(residues_file_median, pocket_file_median, pdb_file, pocket_id, "/Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc")

✅ Config file created: /Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc/config_3hvc_pocket_1.txt


'/Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc/config_3hvc_pocket_1.txt'

In [32]:
create_config_ligand_based(pdb_file, "/Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc")

✅ Config files created: {'A_GG5_361': '/Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc/config_3hvc_ligand_A_GG5_361.txt', 'A_GG5_362': '/Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc/config_3hvc_ligand_A_GG5_362.txt'}


{'A_GG5_361': '/Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc/config_3hvc_ligand_A_GG5_361.txt',
 'A_GG5_362': '/Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc/config_3hvc_ligand_A_GG5_362.txt'}

In [33]:
# # get protein

# import sys
# import os

# # Define the absolute path to the 'src' directory
# src_path = "/Users/nicha/dev/Protein-preparation-pipeline/src"

# # Add 'src' directory to Python path
# if src_path not in sys.path:
#     sys.path.append(src_path)

# print("✅ 'src' directory added to sys.path")


# pdb_id = '3hvc'
# from pdb_retrival.downloader import get_pdb
# from pdb_retrival.pdbfixer import pdbfixer
# from pdb_retrival.pdb_data_retriever import PDBDataRetriever


# from pdbqt_preparation.extract_protein import *
# from pdbqt_preparation.protonation import *
# from pdbqt_preparation.main import *


# pdb_file = get_pdb(pdb_id)
# pdb_fixed_file = pdbfixer(pdb_file)

# print(f"✅ PDB file '{pdb_fixed_file}' downloaded and fixed.")

# # retriever = PDBDataRetriever(pdb_id)
# #     html_content = retriever.fetch_data()
# #     if html_content:
# #         parsed_data = retriever.parse_data(html_content)
# #         retriever.print_data_retriever(parsed_data) 
        
# # main(pdb_fixed_file)


### Calculate the RMDS of ligand poses of 2 different grid box


Run this command to compute RMSD between reference (ref_ligand.pdbqt) and dock poses (dock_ligand.pdbqt):

`obabel ref_ligand.pdbqt dock_ligand.pdbqt --align -xr -O rmsd_output.sdf`

To get RMSD in the terminal:

`obabel ref_ligand.pdbqt dock_ligand.pdbqt --align -xr --rmsd`

In [ ]:
#!/bin/bash

# Exit on errors
set -e

# input path
input_folder=/Users/nicha/dev/Protein-preparation-pipeline/notebooks/rmsd_calc

mkdir -p $input_folder



# Define file paths
VINA_EXEC="vina"  # Path to AutoDock Vina executable (Ensure it is installed & in PATH)
REF_LIGAND="true_ligand.pdb"  # Reference ligand with known binding pose
LIGAND="ligand.pdbqt"  # Input ligand for docking
RECEPTOR="receptor.pdbqt"  # Input receptor for docking
CONFIG1="config_3hvc_pocket_1_medc.txt"  # First docking config file
CONFIG2="config_3hvc_pocket_1_prank.txt"  # Second docking config file
OUT_LIGAND1="docked_ligand1.pdbqt"  # Output for first docking
OUT_LIGAND2="docked_ligand2.pdbqt"  # Output for second docking
CONVERTED1="docked_ligand1.pdb"  # Converted PDB file from first docking
CONVERTED2="docked_ligand2.pdb"  # Converted PDB file from second docking

# Step 1: Run AutoDock Vina for each configuration
echo "🔹 Running Vina with Config 1..."
$VINA_EXEC --receptor $RECEPTOR --ligand $LIGAND --config $CONFIG1 --out $OUT_LIGAND1

echo "🔹 Running Vina with Config 2..."
$VINA_EXEC --receptor $RECEPTOR --ligand $LIGAND --config $CONFIG2 --out $OUT_LIGAND2

# Step 2: Convert PDBQT to PDB (extract first pose)
echo "🔹 Extracting first pose from docked ligands..."
obabel $OUT_LIGAND1 -O $CONVERTED1 --gen3d
obabel $OUT_LIGAND2 -O $CONVERTED2 --gen3d

# Step 3: Calculate RMSD between reference and docked poses
echo "🔹 Calculating RMSD between reference and docked poses..."
RMSD1=$(obabel $REF_LIGAND $CONVERTED1 --align -xr --rmsd | awk '{print $NF}')
RMSD2=$(obabel $REF_LIGAND $CONVERTED2 --align -xr --rmsd | awk '{print $NF}')

# Step 4: Output results
echo "✅ RMSD Results:"
echo "  - RMSD (Config 1 vs Reference): $RMSD1 Å"
echo "  - RMSD (Config 2 vs Reference): $RMSD2 Å"

# Save results to a file
echo -e "Config\tRMSD (Å)" > rmsd_results.txt
echo -e "Config1\t$RMSD1" >> rmsd_results.txt
echo -e "Config2\t$RMSD2" >> rmsd_results.txt

echo "✅ RMSD results saved to rmsd_results.txt"
